In [2]:
import torch
from torch import optim
from torch.nn import functional as F
from torch import nn

import dlc_practical_prologue as prologue

In [3]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        _,_,output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [4]:
def train_model(model, train_input, train_target, train_classes, nb_epochs, mini_batch_size):
    criterion = nn.CrossEntropyLoss()
    eta = 1e-3
    optimizer = optim.Adam(model.parameters(), lr = eta)

    for e in range(nb_epochs):
        acc_loss = 0
        
        for b in range(0, train_input.size(0), mini_batch_size):
            first_digit, second_digit, result = model(train_input.narrow(0, b, mini_batch_size))
            
            loss_digit1 = criterion(first_digit, train_classes[:,0].narrow(0, b, mini_batch_size))
            loss_digit2 = criterion(second_digit, train_classes[:,1].narrow(0, b, mini_batch_size))
            loss_result = criterion(result, train_target.narrow(0, b, mini_batch_size))
            loss_total = loss_result

            acc_loss = acc_loss + loss_total.item()
            model.zero_grad()
            loss_total.backward()
            optimizer.step()
    
        #print(e, acc_loss)

In [18]:
class MLP_WS_NoAL(nn.Module):
    def __init__(self):
        super(MLP_WS_NoAL, self).__init__()
        
        nb_hidden = 100
        input_size = 14*14
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, nb_hidden),
            nn.ReLU(),
            nn.Linear(nb_hidden, nb_hidden),
            nn.ReLU(),
            nn.Linear(nb_hidden, nb_hidden),
            nn.ReLU(),
            nn.Linear(nb_hidden, 10),
            nn.LogSoftmax(dim=1) #Technically this is already in the nn.CrossEntropyLoss()
        )
        
        self.layers_comp = nn.Sequential(
            nn.Linear(20, 200),
            nn.ReLU(),
            nn.Linear(200, 2000),
            nn.ReLU(),
            nn.Linear(2000, 2)
        )
        
    def forward(self, x):
        first_digit = x[:,[0]]
        second_digit = x[:,[1]]
        
        first_digit = first_digit.view(first_digit.size(0),-1) #torch.reshape() can also be used
        second_digit = second_digit.view(second_digit.size(0),-1)
        
        first_digit = self.layers(first_digit)
        second_digit = self.layers(second_digit)
        
        result = torch.cat((first_digit, second_digit), dim=1, out=None)
        result = self.layers_comp(result)
    
        return first_digit, second_digit, result

In [22]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)    

def get_tests(n):
    M = []
    for k in range (0, n):
        L = []
        _, _, _, test_input, test_target, test_classes =  prologue.generate_pair_sets(1000)
        L.append(test_input)
        L.append(test_target)
        L.append(test_classes)
        M.append(L)
    return M

MLP = MLP_WS_NoAL()
nb_epochs = 100
mini_batch_size = 100

train_model(MLP, train_input, train_target, train_classes, nb_epochs, mini_batch_size)

nb_train_errors = compute_nb_errors(MLP, train_input, train_target, mini_batch_size)
print('train error MLP with weight sharing {:0.2f}% {:f}/{:f}'.format((100 * nb_train_errors) / train_input.size(0), nb_train_errors, train_target.size(0)))

L = get_tests(100)
average_nb_test_error = 0
for k in range (0, len(L)):
    nb_test_errors = compute_nb_errors(MLP, L[k][0], L[k][1], mini_batch_size)
    average_nb_test_error += nb_test_errors
    #nb_test_errors = test_accuracy_based_on_result(MLP, L[k][0], L[k][1], mini_batch_size)
    #average_nb_test_error += nb_test_errors
    print('test error MLP {:0.2f}% {:f}/{:f}'.format((100 * nb_test_errors) / L[k][0].size(0), nb_test_errors, L[k][0].size(0)))
print('Average test error MLP with weight sharing {:0.2f}% {:0.1f}/{:d}'.format((100*average_nb_test_error/len(L)) / L[0][0].size(0), average_nb_test_error/len(L), L[0][0].size(0)))

train error MLP with weight sharing 0.00% 0.000000/1000.000000
test error MLP 17.80% 178.000000/1000.000000
test error MLP 15.30% 153.000000/1000.000000
test error MLP 16.20% 162.000000/1000.000000
test error MLP 18.00% 180.000000/1000.000000
test error MLP 13.30% 133.000000/1000.000000
test error MLP 17.20% 172.000000/1000.000000
test error MLP 17.70% 177.000000/1000.000000
test error MLP 15.40% 154.000000/1000.000000
test error MLP 17.10% 171.000000/1000.000000
test error MLP 14.50% 145.000000/1000.000000
test error MLP 16.40% 164.000000/1000.000000
test error MLP 14.60% 146.000000/1000.000000
test error MLP 15.30% 153.000000/1000.000000
test error MLP 16.20% 162.000000/1000.000000
test error MLP 16.50% 165.000000/1000.000000
test error MLP 17.10% 171.000000/1000.000000
test error MLP 17.70% 177.000000/1000.000000
test error MLP 17.10% 171.000000/1000.000000
test error MLP 15.20% 152.000000/1000.000000
test error MLP 17.00% 170.000000/1000.000000
test error MLP 18.20% 182.000000/1000